<a href="https://colab.research.google.com/github/itssofiarce/sagrada/blob/main/codigo/extraccion_estadisticas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!pip install selenium webdriver-manager flask

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3,962 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,535 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd

# Configuración más detallada de las opciones de Chrome
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')

In [ ]:
def obtener_stats(delay=5):
    """
    Obtener las stats, con selenium, de la página de BGG de Sagrada
    args:
        delay: int, tiempo de espera para que cargue la página
    returns:
        pd.DataFrame, tabla con las estadísticas.
    """
    # Configurar Selenium con WebDriver Manager
    driver = webdriver.Chrome(options=chrome_options)

    try:
        url = "https://boardgamegeek.com/boardgame/199561/sagrada/stats"
        driver.get(url)

        print(f"⏳ Esperando {delay} segundos para que carguen los datos...")
        time.sleep(delay)

        stats = driver.find_element(By.CLASS_NAME, "global-body-content-primary.ng-scope") # con s, obtengo una lista, y sin s, un elemento

        # Obtención de los títulos de las estadísticas
        titulos = stats.find_elements(By.CLASS_NAME, "panel-title")
        titulos = [titulo for titulo in titulos if titulo.tag_name == "h3"]
        titulos_texto = [titulo.text for titulo in titulos]

        # Obtención de los valores de las estadísticas (paneles)
        paneles = stats.find_elements(By.CLASS_NAME, "panel-body")

        dic_stadisticos = {}

        for titulo, panel in zip(titulos_texto, paneles):
            if titulo not in ["RATINGS BREAKDOWN"]: # No aporta valor en la columna RATINGS BREAKDOWN
                stats_items = panel.find_elements(By.CLASS_NAME, "outline-item")
                valores = [item.text.replace("\n", " ") for item in stats_items]

                dic_stadisticos[titulo] = valores

    finally:
        driver.quit()
    return pd.DataFrame(dict([(k, pd.Series(v)) for k, v in dic_stadisticos.items()]))

In [ ]:
estadisticas = obtener_stats()
estadisticas.to_csv("sagrada_stats.csv", index=False)
estadisticas

⏳ Esperando 5 segundos para que carguen los datos...


,GAME STATS,GAME RANKS,PLAY STATS,COLLECTION STATS,PARTS EXCHANGE
0,Avg. Rating 7.473,Overall Rank 211 Historical Rank,"All Time Plays 284,311","Own 73,156",Has Parts 19
1,"No. of Ratings 44,031",Abstract Rank 11 Historical Rank,"This Month 1,545","Prev. Owned 4,831",Want Parts 18
2,Std. Deviation 1.16,Family Rank 46 Historical Rank,NaN,For Trade 599 Find For-Trade Matches,NaN
3,Weight 1.92 / 5,NaN,NaN,Want In Trade 832 Find Want-in-Trade Matches,NaN
4,"Comments 6,089",NaN,NaN,"Wishlist 9,260",NaN
5,"Fans 2,184",NaN,NaN,NaN,NaN
6,"Page Views 1,829,210",NaN,NaN,NaN,NaN
